In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


import pandas as pd
import numpy as np
from collections import Counter

n_children = 1000000 # n children to give
n_gift_type = 1000 # n types of gifts available
n_gift_quantity = 1000 # each type of gifts are limited to this quantity
n_gift_pref = 10 # number of gifts a child ranks
n_child_pref = 1000 # number of children a gift ranks
twins = int(0.004 * n_children)    # 0.4% of all population, rounded to the closest even number
ratio_gift_happiness = 2
ratio_child_happiness = 2


gift_pref = pd.read_csv('../input/child_wishlist.csv',header=None).drop(0, 1).values
child_pref = pd.read_csv('../input/gift_goodkids.csv',header=None).drop(0, 1).values


def avg_normalized_happiness(pred, child_pref, gift_pref):
    
    # check if number of each gift exceeds n_gift_quantity
    gift_counts = Counter(elem[1] for elem in pred)
    for count in gift_counts.values():
        assert count <= n_gift_quantity
                
    # check if twins have the same gift
    for t1 in range(0,twins,2):
        twin1 = pred[t1]
        twin2 = pred[t1+1]
        assert twin1[1] == twin2[1]
    
    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for row in pred:
        child_id = row[0]
        gift_id = row[1]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(child_pref[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
    
    # print(max_child_happiness, max_gift_happiness
    print('normalized child happiness=',float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) , \
        ', normalized gift happiness',np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity))
    return float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) + np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity)

random_sub = pd.read_csv('../input/sample_submission_random.csv').values.tolist()
print(avg_normalized_happiness(random_sub, child_pref, gift_pref))

child_wishlist.csv
gift_goodkids.csv
sample_submission_random.csv

normalized child happiness= -0.04404195 , normalized gift happiness 1.18475e-05
-0.0440301025


In [2]:
gift_pref.shape, child_pref.shape

((1000000, 10), (1000, 1000))

In [3]:
class Child(object):
    
    def __init__(self, idx, prefer):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for i in range(prefer.shape[0]):
            self.prefer_dict[prefer[i]] = 400*(prefer.shape[0] - i) - 2
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid] += 2*score + 2
        else:
            self.prefer_dict[giftid] = 2*score - 200
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, -202)

In [4]:
class Child_twin(object):
    
    def __init__(self, idx, prefer1, prefer2):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2))):
            score = 0
            if p in list(prefer1):
                score += 2*(10 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(10 - list(prefer2).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = 100*score - 2
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid] += score + 2
        else:
            self.prefer_dict[giftid] = score - 200
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, -202)

In [5]:
Children = []
for i in range(2000):
    Children.append(Child_twin(2*i, gift_pref[2*i], gift_pref[2*i+1]))
    Children.append(Child_twin(2*i+1, gift_pref[2*i], gift_pref[2*i+1]))
Children = Children + [Child(i, gift_pref[i]) for i in range(4000, 1000000)]

In [6]:
for j in range(1000):
    cf = child_pref[j]
    done_list = []
    for i in range(cf.shape[0]):
        if cf[i] < 4000 and cf[i] not in done_list:
            if cf[i] % 2 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                done_list.append(cid2)
            else:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                done_list.append(cid1)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
        elif cf[i] >= 4000:
            Children[cf[i]].add_gifts_prefer(j, 2*(cf.shape[0] - i))

In [7]:
start_nodes = []
end_nodes = []
capacities = []
unit_costs = []


# twins
for i in range(2000):
    
    for g in Children[2*i].prefer_dict.keys():
        start_nodes.append(g)
        end_nodes.append(i+1000)
        capacities.append(2)
        unit_costs.append(-(Children[2*i].prefer_dict[g] + 202))
        
# non twins
for i in range(4000, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(g)
        end_nodes.append(i-1000)
        capacities.append(1)
        unit_costs.append(-(Children[i].prefer_dict[g] + 202))



In [16]:
min(unit_costs)

-8202

In [77]:
from ortools.graph import pywrapgraph
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

In [78]:
# add Arc
for i in range(len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )

In [79]:
for i in range(1000, 3000):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(i), int(999000), int(2), int(0)
    )
for i in range(4000, 1000000):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(i-1000), int(999000), int(1), int(0)
    )
for i in range(1000):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(i), int(999001), int(1000), int(0)
    )
min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(999001), int(999000), int(1000000), int(0)
    )

11988767

In [80]:
# add Supply
for i in range(1000):
    min_cost_flow.SetNodeSupply(int(i), int(1000))
# twins
for i in range(1000, 3000):
    min_cost_flow.SetNodeSupply(int(i), int(0))
for i in range(4000, 1000000):
    min_cost_flow.SetNodeSupply(int(i-1000), int(0))  
    
min_cost_flow.SetNodeSupply(int(999001), int(0)) 
min_cost_flow.SetNodeSupply(int(999000), int(-1000000)) 

In [81]:
from datetime import datetime

In [82]:
print("Start " ,datetime.now().strftime("%A, %d. %B %Y %I:%M%p"))

Start  Sunday, 17. December 2017 02:49PM


In [83]:
print(min_cost_flow.Solve())

1


In [84]:
print("Finished " ,datetime.now().strftime("%A, %d. %B %Y %I:%M%p"))

Finished  Sunday, 17. December 2017 02:50PM


In [85]:
min_cost_flow.NumArcs()

11988768

In [86]:
res_list_1 = []
for i in range(min_cost_flow.NumArcs()):
    if min_cost_flow.Flow(i) != 0 and min_cost_flow.Head(i) < 999000:
        res_list_1.append([min_cost_flow.Tail(i), min_cost_flow.Head(i), min_cost_flow.Flow(i)])

In [87]:
len(res_list_1)

996489

In [88]:
np.unique([res[2] for res in res_list_1])

array([1, 2])

In [89]:
sum([res[2] for res in res_list_1])

996977

In [90]:
assignment = [-1] * 1000000
for res in res_list_1:
    if res[1] >= 3000:
        if res[2] == 1:
            assignment[res[1]+1000] = res[0]
    else:
        if res[2] == 1:
            if assignment[2*(res[1] - 1000)] != -1:
                if assignment[2*(res[1] - 1000) + 1] != -1:
                    print("Wow")
                else:
                    assignment[2*(res[1] - 1000) + 1] = res[0]
            else:
                assignment[2*(res[1] - 1000)] = res[0]
                
        elif res[2] == 2:
            assignment[2*(res[1] - 1000)] = res[0]
            assignment[2*(res[1] - 1000) + 1] = res[0]

In [91]:
sum([Children[i].happiness(assignment[i]) for i in range(1000000)])

3770156062

In [92]:
3770156062 / 4000000000

0.9425390155

In [95]:
Gifts_counts_left = [1000 for i in range(1000)]
for i in range(1000000):
    if assignment[i] >= 0:
        Gifts_counts_left[assignment[i]] -= 1
Gifts_left = []
for i in range(1000):
    for c in range(Gifts_counts_left[i]):
        Gifts_left.append(i)

In [98]:
# assign
assignment_ = assignment.copy()
for i in range(1000000):
    if assignment_[i] < 0:
        assignment_[i] = Gifts_left.pop()

In [99]:
Gifts_left

[]

In [100]:
res_list_temp = [[i, assignment_[i]] for i in range(1000000)]

In [101]:
avg_normalized_happiness_(res_list_temp, child_pref, gift_pref)

normalized child happiness= 0.8704796 , normalized gift happiness 0.0720599405


0.94253954049999999

In [102]:
Gifts_list_2 = [[] for j in range(1000)]
for res in res_list_temp:
    Gifts_list_2[res[1]].append(res[0])
    
# modify for twins
gain_move = 0
for i in range(2000):
    k1 = res_list_temp[2*i][1]
    k2 = res_list_temp[2*i+1][1]
    # which to go
    if k1 == k2:
        pass
    else:
        # 2*i move
        gain1 = Children[2*i].happiness(k2) - Children[2*i].happiness(k1)
        gain1_add = -10000000
        for l in Gifts_list_2[k2]:
            gain1_add_ = Children[l].happiness(k1) - Children[l].happiness(k2)
            if gain1_add_ > gain1_add and l > 2*i+1:
                v1 = l
                gain1_add = gain1_add_
        # 2*i+1 move
        gain2 = Children[2*i].happiness(k1) - Children[2*i].happiness(k2)
        gain2_add = -10000000
        for l in Gifts_list_2[k1]:
            gain2_add_ = Children[l].happiness(k2) - Children[l].happiness(k1)
            if gain2_add_ > gain2_add and l > 2*i+1:
                v2 = l
                gain2_add = gain2_add_
                
        if gain1 + gain1_add >= gain2 + gain2_add:
            res_list_temp[2*i][1] = k2
            res_list_temp[v1][1] = k1
            Gifts_list_2[k1].remove(2*i)
            Gifts_list_2[k2].append(2*i)
            Gifts_list_2[k2].remove(v1)
            Gifts_list_2[k1].append(v1)
        else:
            res_list_temp[2*i+1][1] = k1
            res_list_temp[v2][1] = k2
            Gifts_list_2[k2].remove(2*i+1)
            Gifts_list_2[k1].append(2*i+1)
            Gifts_list_2[k1].remove(v2)
            Gifts_list_2[k2].append(v2)
        gain_move += max(gain1 + gain1_add, gain2 + gain2_add)
print(gain_move)



-2714


In [103]:
avg_normalized_happiness(res_list_temp, child_pref, gift_pref)

normalized child happiness= 0.8704796 , normalized gift happiness 0.072058737


0.942538337

In [104]:
out = open('../submit/subm_hrd1217_math_harada_2.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(len(res_list_temp)):
    out.write(str(i) + ',' + str(res_list_temp[i][1]) + '\n')
out.close()

In [42]:
Gifts = [[] for g in range(1000)]

In [44]:
sum([sum([Children[c].happiness(j) for c in Gifts[j]]) for j in range(1000)])

3766891080

In [103]:
sum([sum([Children[c].happiness(j) for c in Gifts[j]]) for j in range(1000)])

3766891080

In [104]:
3766891080 / 4000000000

0.94172277

In [112]:
sum([len(g) for g in Gifts])

1000000

In [118]:
res_list_temp = [[] for i in range(1000000)]
for j in range(1000):
    for i in Gifts[j]:
        if len(res_list_temp[i]) > 0:
            # select which to get
            j0 = res_list_temp[i][1]
            if Children[i].happiness(j) > Children[i].happiness(j0):
                res_list_temp[i] = [i, j]
            else:
                pass
        else:
            res_list_temp[i] = [i, j]

In [119]:
len(res_list_temp)

1000000

In [120]:
sum([len(r) for r in res_list_temp])

1999988

In [122]:
sum([Children[r[0]].happiness(r[1]) for r in res_list_temp if len(r) > 0])

3766880892

In [124]:
3766880892 / 4000000000

0.941720223

In [127]:
Gifts_counts = [1000 for i in range(1000)]
for r in res_list_temp:
    if len(r) > 0:
        Gifts_counts[r[1]] -= 1

In [128]:
gifts_left = []
for i in range(1000):
    for _ in range(Gifts_counts[i]):
        gifts_left.append(i)

In [129]:
gifts_left

[555, 691, 728, 866, 922, 924]

In [130]:
for i in range(len(res_list_temp)):
    if len(res_list_temp[i]) == 0:
        res_list_temp[i] = [i, gifts_left.pop()]

In [55]:
def avg_normalized_happiness_(pred, child_pref, gift_pref):
    
    # check if number of each gift exceeds n_gift_quantity
    gift_counts = Counter(elem[1] for elem in pred)
    for count in gift_counts.values():
        assert count <= n_gift_quantity
                
    # check if twins have the same gift
    # for t1 in range(0,twins,2):
    #     twin1 = pred[t1]
    #     twin2 = pred[t1+1]
    #     assert twin1[1] == twin2[1]
    
    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for row in pred:
        child_id = row[0]
        gift_id = row[1]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(child_pref[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
    
    # print(max_child_happiness, max_gift_happiness
    print('normalized child happiness=',float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) , \
        ', normalized gift happiness',np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity))
    return float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) + np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity)





In [134]:
avg_normalized_happiness_(res_list_temp, child_pref, gift_pref)

normalized child happiness= 0.86939285 , normalized gift happiness 0.07232992


0.94172276999999993

In [136]:
Gifts_list_2 = [[] for j in range(1000)]
for res in res_list_temp:
    Gifts_list_2[res[1]].append(res[0])
    
# modify for twins
gain_move = 0
for i in range(2000):
    k1 = res_list_temp[2*i][1]
    k2 = res_list_temp[2*i+1][1]
    # which to go
    if k1 == k2:
        pass
    else:
        # 2*i move
        gain1 = Children[2*i].happiness(k2) - Children[2*i].happiness(k1)
        gain1_add = -10000000
        for l in Gifts_list_2[k2]:
            gain1_add_ = Children[l].happiness(k1) - Children[l].happiness(k2)
            if gain1_add_ > gain1_add and l > 2*i+1:
                v1 = l
                gain1_add = gain1_add_
        # 2*i+1 move
        gain2 = Children[2*i].happiness(k1) - Children[2*i].happiness(k2)
        gain2_add = -10000000
        for l in Gifts_list_2[k1]:
            gain2_add_ = Children[l].happiness(k2) - Children[l].happiness(k1)
            if gain2_add_ > gain2_add and l > 2*i+1:
                v2 = l
                gain2_add = gain2_add_
                
        if gain1 + gain1_add >= gain2 + gain2_add:
            res_list_temp[2*i][1] = k2
            res_list_temp[v1][1] = k1
            Gifts_list_2[k1].remove(2*i)
            Gifts_list_2[k2].append(2*i)
            Gifts_list_2[k2].remove(v1)
            Gifts_list_2[k1].append(v1)
        else:
            res_list_temp[2*i+1][1] = k1
            res_list_temp[v2][1] = k2
            Gifts_list_2[k2].remove(2*i+1)
            Gifts_list_2[k1].append(2*i+1)
            Gifts_list_2[k1].remove(v2)
            Gifts_list_2[k2].append(v2)
        gain_move += max(gain1 + gain1_add, gain2 + gain2_add)
print(gain_move)



8100


In [137]:
avg_normalized_happiness(res_list_temp, child_pref, gift_pref)

normalized child happiness= 0.86939245 , normalized gift happiness 0.07232992


0.94172236999999992

In [138]:
out = open('../submit/subm_hrd1217_math_harada.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(len(res_list_temp)):
    out.write(str(i) + ',' + str(res_list_temp[i][1]) + '\n')
out.close()

In [143]:
sum([Children[r[0]].happiness(r[1]) for r in res_list_temp if len(r) > 0])

3766889480

In [144]:
3766889480 / 4000000000

0.94172237

In [145]:
print("Start " ,datetime.now().strftime("%A, %d. %B %Y %I:%M%p"))

Start  Sunday, 17. December 2017 12:46PM


In [149]:
print(min_cost_flow.SolveMaxFlowWithMinCost())
print("Finished " ,datetime.now().strftime("%A, %d. %B %Y %I:%M%p"))

1
Finished  Sunday, 17. December 2017 12:49PM


In [147]:
res_list_2 = []
for i in range(min_cost_flow.NumArcs()):
    if min_cost_flow.Flow(i) != 0:
        res_list_2.append([min_cost_flow.Tail(i), min_cost_flow.Head(i), min_cost_flow.Flow(i)])

In [148]:
len(res_list_2)

998006

In [150]:
sum([res[2] for res in res_list_1])

1000000

In [151]:
Gifts = [[] for g in range(1000)]
for res in res_list_1:
    if res[1] >= 3000:
        if res[2] == 1:
            Gifts[res[0]].append(res[1] + 1000)
    else:
        if res[2] == 1:
            Gifts[res[0]].append(2*(res[1] - 1000))
        elif res[2] == 2:
            Gifts[res[0]].append(2*(res[1] - 1000))
            Gifts[res[0]].append(2*(res[1] - 1000) + 1)

In [152]:
sum([sum([Children[c].happiness(j) for c in Gifts[j]]) for j in range(1000)])

3766891080

In [153]:
3766891080 / 4000000000

0.94172277

In [100]:
Gifts[:10]

[[1900000,
  365630,
  61280,
  1604366,
  1901908,
  1235478,
  167902,
  1220872,
  456098,
  1501146,
  260408,
  74448,
  1145094,
  1668382,
  430070,
  1019350,
  571930,
  1927448,
  1049948,
  899218,
  520946,
  455896,
  998878,
  732946,
  1194450,
  1065100,
  923540,
  1859626,
  178046,
  1044464,
  1464618,
  1500682,
  378470,
  1369862,
  807414,
  18494,
  1942412,
  687610,
  326456,
  1053462,
  1145530,
  1970502,
  959068,
  349538,
  1750370,
  1381068,
  1286512,
  1274626,
  920526,
  813012,
  1125250,
  778326,
  38866,
  310318,
  1433128,
  905798,
  1692306,
  1502090,
  1825952,
  693644,
  1812618,
  1905272,
  721026,
  324290,
  25614,
  1613864,
  1721100,
  788014,
  370480,
  1750326,
  672106,
  219534,
  1977036,
  1537208,
  1779196,
  1664454,
  1268436,
  929636,
  1891172,
  1050110,
  300450,
  1254922,
  1768150,
  860058,
  246466,
  1855466,
  1760220,
  168228,
  1753444,
  596190,
  1313918,
  1839086,
  647004,
  224736,
  370654,
  119

In [56]:
min_cost_flow.AddArcWithCapacityAndUnitCost(
    start_nodes[ind[0]], end_nodes[ind[0]], capacities[ind[0]], unit_costs[ind[0]]
)

TypeError: in method 'SimpleMinCostFlow_AddArcWithCapacityAndUnitCost', argument 2 of type 'operations_research::NodeIndex'

In [62]:
min_cost_flow.AddArcWithCapacityAndUnitCost(
    int(start_nodes[ind[0]]), end_nodes[ind[0]], capacities[ind[0]], unit_costs[ind[0]]
)

0

In [57]:
start_nodes[ind[0]], end_nodes[ind[0]], capacities[ind[0]], unit_costs[ind[0]]

(0, 924240, 1, -1800)

In [51]:
i

0

In [54]:
start_nodes[ind[0]]

0

In [55]:
end_nodes[ind[0]]

924240

In [40]:
start_nodes = [0,0,1,1,1,2,2,3,4]
end_nodes = [1,2,2,3,4,3,4,4,2]
capacities = [15,8,20,4,10,15,5,20,4]
unit_costs = [4,4,2,2,6,1,3,2,3]

In [17]:
capacities[:10]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [18]:
unit_costs[:10]

[-500, -700, -1813, -1300, -1300, -300, -1900, -433, -700, -900]

In [30]:
min_cost_flow.NumArcs()

0

In [8]:
Children[0].prefer_dict

{90: 231,
 152: 98,
 156: 898,
 220: 498,
 226: 1298,
 229: 698,
 281: 698,
 345: 1098,
 352: 1763,
 409: 1698,
 421: 1698,
 423: 1298,
 498: 1498,
 547: 1498,
 649: 1611,
 660: 1098,
 679: 898,
 682: 709,
 692: 98,
 749: 298,
 834: 298,
 841: 1439,
 871: 1898,
 900: 498,
 929: 1898}